In [1]:
import optuna
import tensorflow as tf
from sklearn.metrics import f1_score
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [3]:
import pandas as pd

In [4]:
train_df = pd.read_csv("train.csv")
train_df

,Unnamed: 0,file_name,label
0,0,train_data/a6dcb93f596a43249135678dfcfc17ea.jpg,1
1,1,train_data/041be3153810433ab146bc97d5af505c.jpg,0
2,2,train_data/615df26ce9494e5db2f70e57ce7a3a4f.jpg,1
3,3,train_data/8542fe161d9147be8e835e50c0de39cd.jpg,0
4,4,train_data/5d81fa12bc3b4cea8c94a6700a477cf2.jpg,1
...,...,...,...
79945,79945,train_data/9283b107f6274279b6f15bbe77c523aa.jpg,0
79946,79946,train_data/4c6b17fe6dd743428a45773135a10508.jpg,1
79947,79947,train_data/1ccbf96d04e342fd9f629ad55466b29e.jpg,0
79948,79948,train_data/ff960b55f296445abb3c5f304b52e104.jpg,1


In [5]:
train_df.label = train_df.label.map({1: 'ai', 0: 'human'})
train_df

,Unnamed: 0,file_name,label
0,0,train_data/a6dcb93f596a43249135678dfcfc17ea.jpg,ai
1,1,train_data/041be3153810433ab146bc97d5af505c.jpg,human
2,2,train_data/615df26ce9494e5db2f70e57ce7a3a4f.jpg,ai
3,3,train_data/8542fe161d9147be8e835e50c0de39cd.jpg,human
4,4,train_data/5d81fa12bc3b4cea8c94a6700a477cf2.jpg,ai
...,...,...,...
79945,79945,train_data/9283b107f6274279b6f15bbe77c523aa.jpg,human
79946,79946,train_data/4c6b17fe6dd743428a45773135a10508.jpg,ai
79947,79947,train_data/1ccbf96d04e342fd9f629ad55466b29e.jpg,human
79948,79948,train_data/ff960b55f296445abb3c5f304b52e104.jpg,ai


In [6]:
train_data = img_gen.flow_from_dataframe(
    train_df,
    x_col='file_name',
    y_col='label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    subset="training"
)

Found 63960 validated image filenames belonging to 2 classes.


In [7]:
val_data = img_gen.flow_from_dataframe(
    train_df,
    x_col='file_name',
    y_col='label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    subset="validation"
)

Found 15990 validated image filenames belonging to 2 classes.


In [8]:
base_model = EfficientNetB0(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

base_model.trainable = True


unfreeze_layers = 0  
for layer in base_model.layers[:-unfreeze_layers]:
    layer.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  
    
    layers.Dense(640, kernel_initializer="he_normal"),
    layers.LeakyReLU(negative_slope=0.1),
    layers.BatchNormalization(),
    layers.Dropout(0.3359853061113047),

    layers.Dense(256, kernel_initializer="he_normal", activation="mish"),
    layers.BatchNormalization(),

    layers.Dense(512, activation='relu', kernel_initializer="he_normal"),

    layers.Dense(1, activation='sigmoid')
])

In [9]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[
        keras.metrics.BinaryAccuracy(),
        keras.metrics.FalseNegatives(),
    ],
)

In [11]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_f1_score', mode='max', save_best_only=True)
]


model.fit(
    train_data,
    validation_data=val_data,
    epochs=7,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/7
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 4781s 2s/step - binary_accuracy: 0.9477 - false_negatives: 650.1650 - loss: 0.1344 - val_binary_accuracy: 0.5073 - val_false_negatives: 7572.0000 - val_loss: 0.7296
Epoch 2/7


D:\.python_projects\ai_vs_human\.venv3\lib\site-packages\keras\src\callbacks\model_checkpoint.py:209: UserWarning: Can save best model only with val_f1_score available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


1999/1999 ━━━━━━━━━━━━━━━━━━━━ 4548s 2s/step - binary_accuracy: 0.9905 - false_negatives: 154.3520 - loss: 0.0294 - val_binary_accuracy: 0.6730 - val_false_negatives: 493.0000 - val_loss: 0.6669
Epoch 3/7
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 4532s 2s/step - binary_accuracy: 0.9930 - false_negatives: 114.6000 - loss: 0.0221 - val_binary_accuracy: 0.5008 - val_false_negatives: 2.0000 - val_loss: 6.2809
Epoch 4/7
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 4525s 2s/step - binary_accuracy: 0.9934 - false_negatives: 112.9530 - loss: 0.0198 - val_binary_accuracy: 0.5000 - val_false_negatives: 7995.0000 - val_loss: 2.8229
Epoch 5/7
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 4539s 2s/step - binary_accuracy: 0.9937 - false_negatives: 97.6270 - loss: 0.0184 - val_binary_accuracy: 0.4999 - val_false_negatives: 7995.0000 - val_loss: 6.1083
Epoch 6/7
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 4765s 2s/step - binary_accuracy: 0.9957 - false_negatives: 73.7900 - loss: 0.0130 - val_binary_accuracy: 0.7053 - val_false_negatives: 48.0000 - val_l